# Use case of PSyKI

This is a brief demonstration of PSyKI.
All steps of the symbolic knowledge injection workflow will be addressed.
We will use some well known datasets for classification tasks for the demonstration.

Importing injectors and utility functions.

In [1]:
from psyki.logic.datalog.grammar.adapters.antlr4 import get_formula_from_string
from psyki.ski.injectors import NetworkComposer, LambdaLayer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.framework.random_seed import set_random_seed
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Input
from test.resources.rules import get_rules

## IRIS
### Iris dataset

In [2]:
SEED = 0
set_random_seed(SEED)

x, y = load_iris(return_X_y=True, as_frame=True)
encoder = OneHotEncoder(sparse=False)
encoder.fit_transform([y])
dataset = x.join(y)
train, test = train_test_split(dataset, test_size=0.5, random_state=SEED)
train_x, train_y = train.iloc[:, :-1], train.iloc[:, -1]
test_x, test_y = test.iloc[:, :-1], test.iloc[:, -1]

train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
3,4.6,3.1,1.5,0.2,0
149,5.9,3.0,5.1,1.8,2
98,5.1,2.5,3.0,1.1,1
6,4.6,3.4,1.4,0.3,0
68,6.2,2.2,4.5,1.5,1
...,...,...,...,...,...
9,4.9,3.1,1.5,0.1,0
103,6.3,2.9,5.6,1.8,2
67,5.8,2.7,4.1,1.0,1
117,7.7,3.8,6.7,2.2,2


### Iris knowledge

In [3]:
class_mapping = {'setosa': 0, 'virginica': 1, 'versicolor': 2}
variable_mapping = {'PL': 0, 'PW': 1, 'SL': 2, 'SW': 3}
formulae = [get_formula_from_string(rule) for rule in get_rules('iris')]

for formula in formulae:
    print(formula)

class(PL,PW,SL,SW,setosa)←((PL)≤(2.28))
class(PL,PW,SL,SW,virginica)←((((PL)>(2.28)))∧(((PW)>(1.64))))
class(PL,PW,SL,SW,versicolor)←((((PL)>(2.28)))∧(((PW)≤(1.64))))


### Iris Predictor

In [4]:
input_layer = Input((4,))
x = Dense(16, activation='relu')(input_layer)
x = Dense(16, activation='relu')(x)
x = Dense(3, activation='softmax')(x)

predictor = Model(input_layer, x)
predictor.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictor.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 16)                80        
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 3)                 51        
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


### Injection via Lambda Layer

In [5]:
injector = LambdaLayer(predictor, class_mapping, variable_mapping)
constrained_predictor = injector.inject(formulae)
constrained_predictor.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
constrained_predictor.summary()

Model: "constrained_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 16)           80          ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 16)           272         ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 3)            51          ['dense_1[0][0]']                
                                                                                  

### Training

In [6]:
constrained_predictor.fit(train_x, train_y, batch_size=8, epochs=50, verbose=1)

Epoch 1/50


2022-06-29 09:55:55.379467: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 0s 977us/step - loss: 1.1101 - accuracy: 0.7333
Epoch 2/50
10/10 [==============================] - 0s 833us/step - loss: 0.9166 - accuracy: 0.7333
Epoch 3/50
10/10 [==============================] - 0s 1ms/step - loss: 0.7567 - accuracy: 0.7333
Epoch 4/50
10/10 [==============================] - 0s 884us/step - loss: 0.6218 - accuracy: 0.7333
Epoch 5/50
10/10 [==============================] - 0s 732us/step - loss: 0.5302 - accuracy: 0.7600
Epoch 6/50
10/10 [==============================] - 0s 1ms/step - loss: 0.4515 - accuracy: 0.7867
Epoch 7/50
10/10 [==============================] - 0s 881us/step - loss: 0.3964 - accuracy: 0.8400
Epoch 8/50
10/10 [==============================] - 0s 854us/step - loss: 0.3573 - accuracy: 0.9200
Epoch 9/50
10/10 [==============================] - 0s 766us/step - loss: 0.3305 - accuracy: 0.9333
Epoch 10/50
10/10 [==============================] - 0s 966us/step - loss: 0.3079 - accuracy: 0.9467
Epoch 11/50
10

### Removing constraints and testing

In [7]:
new_predictor = constrained_predictor.remove_constraints()
new_predictor.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
loss, accuracy = new_predictor.evaluate(test_x, test_y)

accuracy

3/3 [==============================] - 0s 663us/step - loss: 0.1379 - accuracy: 0.9733


0.9733333587646484

### Injection via Network Composer

In [8]:
injector = NetworkComposer(predictor, variable_mapping, layer=2)
structured_predictor = injector.inject(formulae)
structured_predictor.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
structured_predictor.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 lambda_8 (Lambda)              (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 dense_19 (Dense)               (None, 1)            5           ['input_1[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 1)            0           ['input_1[0][0]']                
                                                                                            

### Training

In [9]:
structured_predictor.fit(train_x, train_y, batch_size=8, epochs=50, verbose=1)

Epoch 1/50
10/10 [==============================] - 0s 666us/step - loss: 1.1861 - accuracy: 0.2400
Epoch 2/50
10/10 [==============================] - 0s 709us/step - loss: 1.0312 - accuracy: 0.1867
Epoch 3/50
10/10 [==============================] - 0s 702us/step - loss: 0.9268 - accuracy: 0.0933
Epoch 4/50
10/10 [==============================] - 0s 647us/step - loss: 0.8617 - accuracy: 0.5600
Epoch 5/50
10/10 [==============================] - 0s 620us/step - loss: 0.8060 - accuracy: 0.7200
Epoch 6/50
10/10 [==============================] - 0s 654us/step - loss: 0.7629 - accuracy: 0.7333
Epoch 7/50
10/10 [==============================] - 0s 579us/step - loss: 0.7265 - accuracy: 0.7333
Epoch 8/50
10/10 [==============================] - 0s 719us/step - loss: 0.6892 - accuracy: 0.7333
Epoch 9/50
10/10 [==============================] - 0s 621us/step - loss: 0.6550 - accuracy: 0.7333
Epoch 10/50
10/10 [==============================] - 0s 589us/step - loss: 0.6203 - accuracy: 0.7333

### Testing

In [10]:
loss, accuracy = structured_predictor.evaluate(test_x, test_y)

accuracy

3/3 [==============================] - 0s 878us/step - loss: 0.1993 - accuracy: 0.9867


0.9866666793823242

### Without prior knowledge

In [11]:
predictor.fit(train_x, train_y, batch_size=8, epochs=50, verbose=1)
loss, accuracy = predictor.evaluate(test_x, test_y)

accuracy

Epoch 1/50
10/10 [==============================] - 0s 527us/step - loss: 1.0727 - accuracy: 0.7333
Epoch 2/50
10/10 [==============================] - 0s 585us/step - loss: 0.8754 - accuracy: 0.7333
Epoch 3/50
10/10 [==============================] - 0s 776us/step - loss: 0.7173 - accuracy: 0.7333
Epoch 4/50
10/10 [==============================] - 0s 892us/step - loss: 0.5899 - accuracy: 0.7600
Epoch 5/50
10/10 [==============================] - 0s 442us/step - loss: 0.5031 - accuracy: 0.8533
Epoch 6/50
10/10 [==============================] - 0s 377us/step - loss: 0.4276 - accuracy: 0.9067
Epoch 7/50
10/10 [==============================] - 0s 484us/step - loss: 0.3794 - accuracy: 0.9067
Epoch 8/50
10/10 [==============================] - 0s 424us/step - loss: 0.3436 - accuracy: 0.9067
Epoch 9/50
10/10 [==============================] - 0s 343us/step - loss: 0.3192 - accuracy: 0.9200
Epoch 10/50
10/10 [==============================] - 0s 358us/step - loss: 0.2993 - accuracy: 0.9467

0.9866666793823242

### Observations

In the previous examples we injected three logic rules into a sub-symbolic predictor (a neural network) with two different techniques.
Both techniques have similar performance w.r.t. the same predictor trained without the usage of prior knowledge.
The main reason is that the task is far too simple for a neural network and additional knowledge is not needed.
Let's see the injection in a more challenging task.

## Primate splice-junction gene sequences dataset

In [12]:
from test.resources.data.splice_junction import get_indices
from test.resources.data import get_dataset_dataframe

original_data = get_dataset_dataframe("splice_junction")
original_data.columns = get_indices() + ['target',]
original_data

,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,...,22,23,24,25,26,27,28,29,30,target
0,c,c,a,g,c,t,g,c,a,t,...,g,c,c,a,g,t,c,t,g,ei
1,a,g,a,c,c,c,g,c,c,g,...,t,g,c,c,c,c,c,g,c,ei
2,g,a,g,g,t,g,a,a,g,g,...,a,c,g,g,g,g,a,t,g,ei
3,g,g,g,c,t,g,c,g,t,t,...,g,t,t,t,t,c,c,c,c,ei
4,g,c,t,c,a,g,c,c,c,c,...,c,t,t,g,a,c,c,c,t,ei
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,t,c,t,c,t,t,c,c,c,t,...,t,c,c,t,c,t,c,t,t,n
3186,g,a,g,c,t,c,c,c,a,g,...,g,c,a,c,a,g,c,t,g,n
3187,t,c,t,c,g,g,g,g,g,c,...,g,t,g,t,g,t,g,t,c,n
3188,a,t,t,c,t,a,c,t,t,a,...,c,c,a,a,a,a,c,a,a,n


In [13]:
from test.resources.data import get_splice_junction_processed_dataset

data = get_splice_junction_processed_dataset('data')
train, test = train_test_split(data, train_size=2/3, random_state=SEED, stratify=data.iloc[:, -1])
train_x, train_y = train.iloc[:, :-1], train.iloc[:, -1:]
test_x, test_y = test.iloc[:, :-1], test.iloc[:, -1:]
data

,0,1,2,3,4,5,6,7,8,9,...,231,232,233,234,235,236,237,238,239,240
0,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
1,1,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,2
3186,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,2
3187,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,2
3188,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,2


### Splice-junction knowledge

In [14]:
from test.resources.data import get_splice_junction_extended_feature_mapping
from test.resources.rules import get_splice_junction_formulae

variable_mapping = get_splice_junction_extended_feature_mapping()
formulae = get_splice_junction_formulae('kb')

### Splice-junction predictor

In [15]:
input_layer = Input((240,))
x = Dense(64, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(3, activation='softmax')(x)

predictor = Model(input_layer, x)
predictor.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictor.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 240)]             0         
                                                                 
 dense_27 (Dense)            (None, 64)                15424     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_28 (Dense)            (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_29 (Dense)            (None, 3)                 99        
                                                                 
Total params: 17,603
Trainable params: 17,603
Non-trainable

### Injection via Network Composer

In [16]:
injector = NetworkComposer(predictor, variable_mapping)
structured_predictor = injector.inject(formulae)
structured_predictor.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [17]:
structured_predictor.fit(train_x, train_y, epochs=30, batch_size=32, verbose=1)

Epoch 1/30
67/67 [==============================] - 5s 4ms/step - loss: 0.8383 - accuracy: 0.6082
Epoch 2/30
67/67 [==============================] - 0s 6ms/step - loss: 0.4197 - accuracy: 0.8410
Epoch 3/30
67/67 [==============================] - 0s 7ms/step - loss: 0.2632 - accuracy: 0.9083
Epoch 4/30
67/67 [==============================] - 0s 7ms/step - loss: 0.1857 - accuracy: 0.9351
Epoch 5/30
67/67 [==============================] - 0s 7ms/step - loss: 0.1547 - accuracy: 0.9548
Epoch 6/30
67/67 [==============================] - 0s 7ms/step - loss: 0.1329 - accuracy: 0.9610
Epoch 7/30
67/67 [==============================] - 0s 7ms/step - loss: 0.1157 - accuracy: 0.9638
Epoch 8/30
67/67 [==============================] - 0s 7ms/step - loss: 0.0998 - accuracy: 0.9699
Epoch 9/30
67/67 [==============================] - 0s 6ms/step - loss: 0.0775 - accuracy: 0.9821
Epoch 10/30
67/67 [==============================] - 0s 6ms/step - loss: 0.0717 - accuracy: 0.9817
Epoch 11/30
67/67 [

### Testing

In [18]:
loss, accuracy = structured_predictor.evaluate(test_x, test_y)

accuracy

34/34 [==============================] - 1s 2ms/step - loss: 0.2194 - accuracy: 0.9408


0.9407894611358643

### Without prior knowledge

In [19]:
predictor.fit(train_x, train_y, epochs=30, batch_size=32, verbose=1)
loss, accuracy = predictor.evaluate(test_x, test_y)

accuracy

Epoch 1/30
67/67 [==============================] - 0s 561us/step - loss: 0.8981 - accuracy: 0.5682
Epoch 2/30
67/67 [==============================] - 0s 532us/step - loss: 0.4561 - accuracy: 0.8354
Epoch 3/30
67/67 [==============================] - 0s 540us/step - loss: 0.2708 - accuracy: 0.9069
Epoch 4/30
67/67 [==============================] - 0s 636us/step - loss: 0.1866 - accuracy: 0.9436
Epoch 5/30
67/67 [==============================] - 0s 527us/step - loss: 0.1651 - accuracy: 0.9483
Epoch 6/30
67/67 [==============================] - 0s 583us/step - loss: 0.1352 - accuracy: 0.9595
Epoch 7/30
67/67 [==============================] - 0s 608us/step - loss: 0.1120 - accuracy: 0.9699
Epoch 8/30
67/67 [==============================] - 0s 521us/step - loss: 0.0893 - accuracy: 0.9751
Epoch 9/30
67/67 [==============================] - 0s 515us/step - loss: 0.0805 - accuracy: 0.9774
Epoch 10/30
67/67 [==============================] - 0s 522us/step - loss: 0.0865 - accuracy: 0.9741

0.9351503849029541